# Main Goals of the project
- predict 'taxvaluedollarcount'
    - on single unit properties
    - between May and June of 2017
- what states and counties are properties located
- plot distribution of tax rates for each county

For the first iteration of your model, use only square feet of the home, number of bedrooms, and number of bathrooms to estimate the property's assessed value, taxvaluedollarcnt. You can expand this to other fields after you have completed an mvp (minimally viable product).

##### initial query
select calculatedfinishedsquarefeet, bedroomcnt, bathroomcnt, taxvaluedollarcnt

from properties_2017 as prop

join predictions_2017 as pred on pred.parcelid = prop.parcelid

where transactiondate BETWEEN '2017-05-01' AND '2017-06-30'

and propertylandusetypeid = 261

and calculatedfinishedsquarefeet is not null

and bedroomcnt > 0

and bathroomcnt > 0;


In [1]:
import pandas as pd
from acquire import get_zillow_data

In [ ]:
df = get